In [1]:
pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gradio as gr
import requests
import re
import os

import config




# 환경 변수에서 키 가져오기
import os
from dotenv import load_dotenv

# .env 파일 로드 (개발 환경용)
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "")
api_key = os.getenv("AZURE_OPENAI_API_KEY", "")
ai_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT", "")
ai_search_api_key = os.getenv("AZURE_SEARCH_API_KEY", "")


ai_search_index = "stardew_valley_index"
ai_search_semantic = "stardew-valley-semantic"

# 캐릭터 이미지 매핑 (캐릭터 이름별 이미지 URL)
character_images = {
    "Pierre": "https://stardewvalleywiki.com/mediawiki/images/7/7e/Pierre.png",
    "Caroline": "https://stardewvalleywiki.com/mediawiki/images/8/87/Caroline.png",
    "Abigail": "https://stardewvalleywiki.com/mediawiki/images/8/88/Abigail.png",
    "Lewis": "https://stardewvalleywiki.com/mediawiki/images/2/2b/Lewis.png",
    "Robin": "https://stardewvalleywiki.com/mediawiki/images/1/1b/Robin.png",
    "Demetrius": "https://stardewvalleywiki.com/mediawiki/images/c/c8/Demetrius.png",
    "Maru": "https://stardewvalleywiki.com/mediawiki/images/9/9e/Maru.png",
    "Sebastian": "https://stardewvalleywiki.com/mediawiki/images/a/a8/Sebastian.png",
    "Linus": "https://stardewvalleywiki.com/mediawiki/images/3/31/Linus.png",
    "Jodi": "https://stardewvalleywiki.com/mediawiki/images/4/41/Jodi.png",
    "Kent": "https://stardewvalleywiki.com/mediawiki/images/9/91/Kent.png",
    "Sam": "https://stardewvalleywiki.com/mediawiki/images/9/94/Sam.png",
    "Vincent": "https://stardewvalleywiki.com/mediawiki/images/f/f1/Vincent.png",
    "Penny": "https://stardewvalleywiki.com/mediawiki/images/a/ab/Penny.png",
    "Pam": "https://stardewvalleywiki.com/mediawiki/images/d/da/Pam.png",
    "Gus": "https://stardewvalleywiki.com/mediawiki/images/5/52/Gus.png",
    "Emily": "https://stardewvalleywiki.com/mediawiki/images/2/28/Emily.png",
    "Haley": "https://stardewvalleywiki.com/mediawiki/images/1/1b/Haley.png",
    "Evelyn": "https://stardewvalleywiki.com/mediawiki/images/8/8e/Evelyn.png",
    "George": "https://stardewvalleywiki.com/mediawiki/images/7/78/George.png",
    "Alex": "https://stardewvalleywiki.com/mediawiki/images/0/04/Alex.png",
    "Shane": "https://stardewvalleywiki.com/mediawiki/images/8/8b/Shane.png",
    "Marnie": "https://stardewvalleywiki.com/mediawiki/images/5/52/Marnie.png",
    "Jas": "https://stardewvalleywiki.com/mediawiki/images/5/55/Jas.png",
    "Clint": "https://stardewvalleywiki.com/mediawiki/images/3/31/Clint.png",
    "Harvey": "https://stardewvalleywiki.com/mediawiki/images/9/95/Harvey.png",
    "Elliott": "https://stardewvalleywiki.com/mediawiki/images/b/bd/Elliott.png",
    "Willy": "https://stardewvalleywiki.com/mediawiki/images/8/82/Willy.png",
    "Wizard": "https://stardewvalleywiki.com/mediawiki/images/c/c7/Wizard.png",
    "Krobus": "https://stardewvalleywiki.com/mediawiki/images/7/71/Krobus.png",
    "Dwarf": "https://stardewvalleywiki.com/mediawiki/images/e/ed/Dwarf.png",
    "Sandy": "https://stardewvalleywiki.com/mediawiki/images/4/4e/Sandy.png",
    "Gunther": "https://stardewvalleywiki.com/mediawiki/images/3/3d/Gunther.png",
    "Leah": "https://stardewvalleywiki.com/mediawiki/images/e/e6/Leah.png",
}

def request_gpt(prompt):
    
    # HTTP 요청에 필요한 헤더 설정  
    headers = {  
        "Content-Type": "application/json",  
        "api-key": api_key
    }
    
    # HTTP 요청의 본문 데이터  
    body = {  
        "messages": [  
            {  
                "role": "system",  
                "content": "마을 촌장님처럼 말투를 구사해줘. 능글맞은 촌장님이라고 생각하고, 각 주민들이 자기와 친하다는 듯한 말투도 구사해줘."
            },  
            {  
                "role": "user",  
                "content": prompt  
            }  
        ],  
        "temperature": 0.7,  
        "top_p": 0.95,  
        "max_tokens": 800,  
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                    "type": "api_key",
                    "key": ai_search_api_key
                },
                "key": ai_search_api_key,
                
            }
            }
        ],
    }  
    
    # POST 요청을 보내고 응답 받기  
    response = requests.post(endpoint, headers=headers, json=body)  

    if response.status_code == 200:
        # 응답을 JSON 형식으로 파싱  
        response_json = response.json()  
        
        # 모델이 생성한 메시지 추출  
        message = response_json['choices'][0]['message']  
        citation_list = message['context']['citations']
        # 역할(role)과 내용(content) 분리  
        role = message['role']  
        content = message['content']  
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        
        # 프롬프트 및 응답에서 캐릭터 이름 감지
        mentioned_characters = []
        for character in character_images.keys():
            if character.lower() in prompt.lower() or character.lower() in content.lower():
                mentioned_characters.append(character)
        
        # 기본 캐릭터를 Lewis(촌장)으로 설정
        character_name = "Lewis"
        
        # 언급된 캐릭터가 있으면 첫 번째 캐릭터로 설정
        if mentioned_characters:
            character_name = mentioned_characters[0]
        
        character_image = character_images.get(character_name, character_images["Lewis"])
        
        return content, citation_list, character_image, character_name
    else:
        return "", list(), character_images["Lewis"], "Lewis"

# 타이틀 HTML
title_html = """
<div id="stardew-title" style="font-family: 'SDMisaeng', sans-serif; color: #f4af3d; text-shadow: 2px 2px 4px #5e3b18; font-size: 36px; text-align: center; margin-bottom: 20px; font-weight: bold;">스타듀밸리 촌장님께 물어보세요!</div>
"""

# 푸터 HTML
footer_html = """
<div id="stardew-footer" style="text-align: center; font-family: 'SDMisaeng', sans-serif; color: #5e3b18; margin-top: 10px; font-size: 14px;">© 2025 스타듀밸리 농장 정보 서비스</div>
"""

# 커스텀 CSS 정의
custom_css = """
.gradio-container {
    background: url('https://wallpapercave.com/wp/wp6659138.jpg');
    background-size: cover;
}
@font-face {
    font-family: 'SDMisaeng'; 
    src: local('SDMisaeng');
}
* {
    font-family: 'SDMisaeng', sans-serif !important;
}
"""

# gr.Blocks에 직접 CSS 설정
with gr.Blocks(css=custom_css, elem_id="stardew-container") as demo:
    gr.HTML(title_html)
    
    with gr.Row():
        # 왼쪽에 챗봇 배치
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="촌장님과의 대화", 
                height=450, 
                elem_id="chatbot-area",
                elem_classes="chatbot-container"
            )
        
        # 오른쪽에 캐릭터 이미지 배치
        with gr.Column(scale=1, elem_id="character-area", elem_classes="character-panel"):
            character_name_display = gr.Textbox(
                label="캐릭터", 
                value="Lewis", 
                elem_id="character-display",
                elem_classes="character-label"
            )
            character_image = gr.Image(
                label="", 
                value=character_images["Lewis"],
                elem_id="character-image"
            )
    
    # 프롬프트 입력 영역
    with gr.Row():
        prompt_textbox = gr.Textbox(
            label='질문 내용', 
            placeholder="스타듀밸리에 대해 물어보세요...", 
            scale=6, 
            elem_id="prompt-input",
            elem_classes="prompt-box"
        )
        send_button = gr.Button(
            '전송', 
            scale=1, 
            elem_id="send-button",
            elem_classes="send-btn"
        )
    
    # 맨 아래에 citation 영역 배치
    citation_textbox = gr.Textbox(
        label="참조 문서", 
        lines=5, 
        elem_id="citations",
        elem_classes="citation-box"
    )
    
    # 푸터 추가
    gr.HTML(footer_html)

    def click_send(prompt, histories, current_character):
        if not prompt.strip():
            return histories, current_character, character_images.get(current_character, character_images["Lewis"]), citation_textbox.value, ""
        
        content, citation_list, char_img, char_name = request_gpt(prompt=prompt)
        
        # Gradio Chatbot을 위한 튜플 형식으로 메시지 구성
        histories.append((prompt, content))
        
        # citation_list를 텍스트로 변환
        citation_text = "\n".join([f"{i+1}. {citation['content']}" for i, citation in enumerate(citation_list)])
        
        return histories, char_name, char_img, citation_text, ""

    send_button.click(
        click_send, 
        inputs=[prompt_textbox, chatbot, character_name_display], 
        outputs=[chatbot, character_name_display, character_image, citation_textbox, prompt_textbox]
    )
    
    # 엔터키 입력 이벤트도 추가
    prompt_textbox.submit(
        click_send,
        inputs=[prompt_textbox, chatbot, character_name_display],
        outputs=[chatbot, character_name_display, character_image, citation_textbox, prompt_textbox]
    )

# 실행 시 브라우저에서 바로 열기
demo.launch(share=True, inbrowser=True)

C:\Users\EL0019\AppData\Local\Temp\ipykernel_25076\2641232540.py:168: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://58d20126db5307234e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
pip install gradio==3.50.2  # 예시 버전

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
